## Welcome to DEM529 Memory and Multi-agent RAG with AI Search Demo



Let's start with reading demo enviroment variables

In [4]:
import os
from mem0 import Memory
from openai import AzureOpenAI
from dotenv import load_dotenv

dotenv_path = load_dotenv(".env")

# Load Azure AI Search configuration
SEARCH_SERVICE_ENDPOINT = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
SEARCH_SERVICE_API_KEY = os.getenv("AZURE_SEARCH_ADMIN_KEY")
SEARCH_SERVICE_NAME= os.getenv("AZURE_SEARCH_SERVICE_NAME")

# Load Azure OpenAI configuration
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME = os.getenv("AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME")
AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME")

# Create Azure OpenAI client
azure_openai_client = AzureOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    api_version="2025-01-01-preview"
)

# Part 1 

Example Myself + ChatGPT 
- Showcases basic interaction with Azure OpenAI

In [5]:
messages = [
    {"role": "system", 
        "content": "You are a helpful travel assistant for travel guidance. "
        "Be concise, specific, and helpful. Refer to the user by name when appropriate. "
        "Recommend specific places when relevant."
    },
    {"role": "user", "content": "Hi, my name is Ben. I need recommendations of nice places that serves beverages near ski lift!"}
]

response = azure_openai_client.chat.completions.create(
        model=AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME,
        messages=messages,
        temperature=0.2,  # Adjust for more or less randomness
        max_tokens=800     # Limit the length of the response
    )

if response.choices:
        print(f"Response: {response.choices[0].message.content}")
else:
        print("No response received.")

Response: Hi Ben! Here are some great places near ski lifts where you can enjoy beverages:

1. **Aspen, Colorado**: Check out the *Ajax Tavern* located at the base of Aspen Mountain. They offer a great selection of beers and cocktails.

2. **Park City, Utah**: *The Corner Store* is a cozy spot near the Park City Mountain Resort ski lift, perfect for grabbing a hot drink or a cold beer.

3. **Whistler, Canada**: Visit *The Longhorn Saloon* at the base of Whistler Mountain for a lively atmosphere and a variety of drinks.

4. **Lake Tahoe, California/Nevada**: *The Loft* at Heavenly offers a great selection of wines and cocktails right by the gondola.

5. **Vail, Colorado**: *The 10th* is a fantastic on-mountain restaurant that serves drinks with stunning views, located near the Eagle Bahn Gondola.

Let me know if you need more information about any of these places!


![alt text](image-1.png)

# Part 2

**Environment Setup**: 
- Azure OpenAI and Azure AI Search configurations are loaded using environment variables.
- The `Memory` class from Mem0 is configured to store and retrieve user-specific memories.

In [6]:
# Basic example for Azure AI Search with Mem0
import os
from mem0 import Memory

# Load Azure configuration
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME= os.getenv("AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME")
AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME")
SEARCH_SERVICE_ENDPOINT = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
SEARCH_SERVICE_API_KEY = os.getenv("SEARCH_SERVICE_API_KEY")
SEARCH_SERVICE_NAME = os.getenv("SEARCH_SERVICE_NAME")

# Configure Mem0 with Azure AI Search
memory_config = {
    "vector_store": {
        "provider": "azure_ai_search",
        "config": {
            "service_name": SEARCH_SERVICE_NAME,
            "api_key": SEARCH_SERVICE_API_KEY,
            "collection_name": "travel_memories",
            "embedding_model_dims": 3072,
        },
    },
    "embedder": {
        "provider": "azure_openai",
        "config": {
            "model": AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME,
            "embedding_dims": 3072,
            "azure_kwargs": {
                "api_version": "2024-12-01-preview",
                "azure_deployment": AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME,
                "azure_endpoint": AZURE_OPENAI_ENDPOINT,
                "api_key": AZURE_OPENAI_API_KEY,
            },
        },
    },
    "llm": {
        "provider": "azure_openai",
        "config": {
            "model": AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME,
            "temperature": 0.1,
            "max_tokens": 800,
            "azure_kwargs": {
                "azure_deployment": AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME, 
                "api_version": "2025-01-01-preview",
                "azure_endpoint": AZURE_OPENAI_ENDPOINT,
                "api_key": AZURE_OPENAI_API_KEY,
            },
        },
    },
    "version": "v1.1",
}

# Initialize memory
memory = Memory.from_config(memory_config)
print("Mem0 initialized with Azure AI Search")

Mem0 initialized with Azure AI Search


**Basic Memory Operations**:
    - Demonstrates adding, searching, and retrieving memories for a user. For example, storing travel preferences or conversations and retrieving them based on queries.


![alt text](image-3.png)

In [7]:
# Basic memory operations
def demonstrate_basic_operations():
    user_id="ben_whistler_2025"
    
    # 1. Add a simple memory
    print("\n===== STORING MEMORIES =====")
    memory.add(
        "I enjoy skiing and outdoor activities. I also like drinking coffee with mountain view.",
        user_id=user_id
    )
    print("✓ Added memory about skiing preferences")
    
    # 2. Add conversation messages
    conversation = [
        {"role": "user", "content": "It is time to plan our annual winter ski trip to Whistler."},
        {"role": "agent", "content": "That's a great time to visit Whistler! The winter is a great time for outdoor activities."},
        {"role": "user", "content": "I like skiing and an expert skier."}
   ]
    memory.add(conversation, user_id=user_id)
    print("✓ Added conversation about Whistler trip")
    
    # 3. Add memory with metadata
    memory.add(
        "Last year, I had an injury, that is why I want to avoid black and advanced ski slopes.",
        user_id=user_id,
        metadata={"category": "past_experience", "importance": "medium"}
    )
    print("✓ Added past experiences with metadata")
    
    # 4. Search memories
    print("\n===== SEARCHING MEMORIES =====")
    search_results = memory.search("What are this user's travel plans?", user_id=user_id, limit=3)
    print(f"Found {len(search_results['results'])} relevant memories:")
    for i, result in enumerate(search_results['results'], 1):
        print(f"{i}. {result['memory']} (Score: {result['score']:.4f})")
    
    # 5. Get all memories
    print("\n===== ALL MEMORIES =====")
    all_memories = memory.get_all(user_id=user_id)
    print(f"Total memories: {len(all_memories['results'])}")
    for i, memory_item in enumerate(all_memories['results'], 1):
        print(f"{i}. {memory_item['memory']}")
    
    return "Basic operations demonstrated successfully!"

# Run the demonstration
result = demonstrate_basic_operations()
print(f"\n{result}")


===== STORING MEMORIES =====
✓ Added memory about skiing preferences
✓ Added conversation about Whistler trip
✓ Added past experiences with metadata

===== SEARCHING MEMORIES =====
Found 3 relevant memories:
1. Planning annual winter ski trip to Whistler (Score: 0.6132)
2. Enjoys skiing and outdoor activities (Score: 0.5782)
3. Likes skiing (Score: 0.5651)

===== ALL MEMORIES =====
Total memories: 6
1. Planning annual winter ski trip to Whistler
2. Likes drinking coffee with mountain view
3. Wants to avoid black and advanced ski slopes due to a past injury
4. Likes skiing
5. Enjoys skiing and outdoor activities
6. Is an expert skier

Basic operations demonstrated successfully!


# Part 3

**Travel Assistant Class**:
    - Implements a `TravelAssistant` class that integrates memory and Azure OpenAI to provide personalized responses.
    - It retrieves relevant memories to include in the context of responses and stores new conversations in memory.

![alt text](image.png)

In [8]:
# Part 2: Travel Planning Assistant with Memory

class TravelAssistant:
    def __init__(self, name:str, user_id, system_message:str | None = "You are a helpful assistant that can respond to messages."):
        """TravelAssistant: Useful for getting travel plans, locations, and suggestions."""
        self.user_id = user_id
        
        # Configure memory for travel planning
        memory_config = {
            "vector_store": {
                "provider": "azure_ai_search",
                "config": {
                    "service_name": SEARCH_SERVICE_NAME,
                    "api_key": SEARCH_SERVICE_API_KEY,
                    "collection_name": "travel_memories",
                    "embedding_model_dims": 3072,
                    "compression_type": "binary",
                },
            },
            "llm": {
                "provider": "azure_openai",
                "config": {
                    "model": AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME,
                    "temperature": 0.7,
                    "max_tokens": 800,
                    "azure_kwargs": {
                        "azure_deployment": AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME,
                        "api_version": "2025-01-01-preview",
                        "azure_endpoint": AZURE_OPENAI_ENDPOINT,
                        "api_key": AZURE_OPENAI_API_KEY,
                    },
                },
            },
            "embedder": {
                "provider": "azure_openai",
                "config": {
                    "model": AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME,
                    "embedding_dims": 3072,
                    "azure_kwargs": {
                        "api_version": "2024-12-01-preview",
                        "azure_deployment": AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME,
                        "azure_endpoint": AZURE_OPENAI_ENDPOINT,
                        "api_key": AZURE_OPENAI_API_KEY,
                    },
                },
            },
            "version": "v1.1",
        }
        
        self.memory = Memory.from_config(memory_config)
        self.azure_client = azure_openai_client
        print(f"Travel Assistant initialized for user {user_id}")
    
    def get_response(self, query: str, memory_context: bool =True) -> str:
        """Get response from Azure OpenAI with memory context"""
        # Retrieve relevant memories if enabled
        memory_text = ""
        if memory_context:
            memories = self.memory.search(query, user_id=self.user_id)
            if 'results' in memories and memories['results']:
                memory_text = "\n\nRelevant information from previous conversations:\n"
                included_memories = []
                for i, mem in enumerate(memories['results'][:5], 1):
                    memory_text += f"{i}. {mem['memory']}\n"
                    included_memories.append(mem['memory'])
                print(f"Including {len(memories['results'][:5])} memories in context")
                print("Included Memories:")
                for memory in included_memories:
                    print(f"- {memory}")
            else:
                print("No relevant memories found")
        
        
        # Construct messages with system prompt and memory context
        messages = [
            {
                "role": "system", 
                "content": "You are a helpful travel assistant for travel guidance. "
                           "Be concise, specific, and helpful. Refer to the user by name when appropriate. "
                           "Recommend specific places when relevant."
            },
            {
                "role": "user", 
                "content": f"{query}\n{memory_text}"
            }
        ]
        
        # Get response from Azure OpenAI
        response = self.azure_client.chat.completions.create(
            model=AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME,
            messages=messages,
            temperature=0.7,
            max_tokens=800,
        )
        
        # Extract response content
        response_content = response.choices[0].message.content
        
        # Store the conversation in memory
        #conversation = [
        #    {"role": "user", "content": query},
        #    {"role": "assistant", "content": response_content}
        #]
        #self.memory.add(conversation, user_id=self.user_id)
        
        return response_content
    

# Part 3b 
Interaction with Travel Assistant using created memories

In [9]:
#Travel Assistant Demo
# This is a simple demo of the Travel Assistant using Azure OpenAI and No Memory

# Create travel assistant for Ben
print("\n\n=== TRAVEL PLANNING ASSISTANT DEMO ===\n")
assistant = TravelAssistant("travel_assistant",user_id="ben_whistler_2025")

# Second interaction - Specific question about coffee places
query1 = "Hi, my name is Ben. I need recommendations of nice places that serves beverages near ski lift!"
print(f"User: {query1}")
response1 = assistant.get_response(query1)  # Should use memory context
print(f"Assistant: {response1}\n")

# Verify what memories have been stored
#assistant.verify_memories()



=== TRAVEL PLANNING ASSISTANT DEMO ===

Travel Assistant initialized for user ben_whistler_2025
User: Hi, my name is Ben. I need recommendations of nice places that serves beverages near ski lift!
Including 5 memories in context
Included Memories:
- Likes drinking coffee with mountain view
- Likes skiing
- Enjoys skiing and outdoor activities
- Wants to avoid black and advanced ski slopes due to a past injury
- Planning annual winter ski trip to Whistler
Assistant: Hi Ben! For your ski trip to Whistler, here are some great places that serve beverages with a nice view near the ski lifts:

1. **The Roundhouse Lodge** - Located at the peak of Whistler Mountain, this lodge has a fantastic view and serves coffee, hot chocolate, and snacks. It's a perfect spot to relax between runs.

2. **Green Room** - This cozy spot is right at the base of Blackcomb Mountain. They offer a variety of coffee drinks and a lovely view of the slopes.

3. **Whistler Coffee Company** - Located in Whistler Villa

# Part 4a

Introduction to Product Advisor assistant

A series of product being stored in AI Search indexes that will be used to retrieve relevant producs from, instead of generic responses from Web.

![alt text](image-5.png)

In [10]:
# Product Advisor with Memory and Hybrid Search for Product Catalog
from autogen_ext.tools.azure._ai_search import AzureAISearchTool, SearchResults

class ProductAdvisor:
    """ProductAdvisor: Useful for getting product recommendations and suggestions."""
    def __init__(self, user_id):
        """Initialize product advisor with memory and hybrid search for product catalog."""
        self.user_id = user_id
        # Configure memory (for conversation history)
        memory_config = {
            "vector_store": {
                "provider": "azure_ai_search",
                "config": {
                    "service_name": SEARCH_SERVICE_NAME,
                    "api_key": SEARCH_SERVICE_API_KEY,
                    "collection_name": "travel_memories",
                    "embedding_model_dims": 3072,
                    "compression_type": "binary",
                },
            },
            "llm": {
                "provider": "azure_openai",
                "config": {
                    "model": AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME,
                    "temperature": 0.7,
                    "max_tokens": 800,
                    "azure_kwargs": {
                        "azure_deployment": AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME,
                        "api_version": "2025-01-01-preview",
                        "azure_endpoint": AZURE_OPENAI_ENDPOINT,
                        "azure_api_key": AZURE_OPENAI_API_KEY,
                    },
                },
            },
            "embedder": {
                "provider": "azure_openai",
                "config": {
                    "model": AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME,
                    "embedding_dims": 3072,
                    "azure_kwargs": {
                        "api_version": "2024-12-01-preview",
                        "azure_deployment": AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME,
                        "azure_endpoint": AZURE_OPENAI_ENDPOINT,
                        "azure_api_key": AZURE_OPENAI_API_KEY,
                    },
                },
            },
            "version": "v1.1",
        }
        self.memory = Memory.from_config(memory_config)
        self.azure_client = azure_openai_client
        print(f"Product Advisor initialized for user {user_id} with hybrid search for product catalog.")

    async def search_product_catalog(self, query: str):
        """Performs hybrid search on the product catalog using Azure AI Search."""
       
        from azure.search.documents import SearchClient
        from azure.core.credentials import AzureKeyCredential
        
        vector_store = AzureAISearchTool.create_hybrid_search(
            name="doc-search",
            endpoint=SEARCH_SERVICE_ENDPOINT,  # Your Azure AI Search endpoint
            index_name="vector-1746814756295",  # Name of your search index
            credential=AzureKeyCredential(SEARCH_SERVICE_API_KEY),  # Your Azure AI Search admin key
            vector_fields= ["text_vector"],
            #query_type="full",  # Enable Lucene query syntax
            search_fields=["chunk", "title"],  # Required: fields to search within
            select_fields=["chunk", "title"],  # Optional: fields to return
            top=3,
        )
        

        try:
            search_results = await vector_store.run(query)
            print("Full text search result:", search_results)
            if hasattr(search_results, "results"):
                results_list = search_results.results
            else:
                results_list = list(search_results) if search_results else []
            return results_list
        finally:
            # If vector_store has a close or aclose method, call it here
            await vector_store.close()
            #close_fn = getattr(vector_store, "close", None) or getattr(vector_store, "aclose", None)
            #if close_fn:
            #    await close_fn()
                    
      
     
    async def get_product_response(self, query: str, memory_context: bool =True, use_product_catalog: bool =True) -> str:
        """Get response from Azure OpenAI with memory and product catalog context."""

        print("Using Product Advisor Agent.")
        # Retrieve relevant memories if enabled
        memory_text = ""
        if memory_context:
            memories = self.memory.search(query, user_id=self.user_id)
            if 'results' in memories and memories['results']:
                memory_text = "\n\nRelevant information from previous conversations:\n"
                for i, mem in enumerate(memories['results'][:5], 1):
                    memory_text += f"{i}. {mem['memory']}\n"
                print(f"Including {len(memories['results'][:5])} memories in context")
            else:
                print("No relevant memories found")
     
        product_catalog_text = ""
        if use_product_catalog:
            try:
                responses = await self.search_product_catalog(query)
                if responses:
                    product_catalog_text = "\n\nRelevant products from the catalog:\n"
                    for i, item in enumerate(responses[:5], 1):
                        # If item is an object, extract a string representation
                        if hasattr(item, "page_content"):
                            product_catalog_text += f"{i}. {item.page_content}\n"
                        elif hasattr(item, "chunk"):
                            product_catalog_text += f"{i}. {item.chunk}\n"
                        else:
                            product_catalog_text += f"{i}. {str(item)}\n"
                    print(f"Including {len(responses[:5])} product responses in context")
                else:
                    product_catalog_text = "No relevant products found in the catalog."
                    print(product_catalog_text)
            except Exception as e:
                product_catalog_text = f"Error searching product catalog: {e}"
                print(product_catalog_text)

        # Construct messages with system prompt, memory context, and product catalog context
        messages = [
            {
                "role": "system",
                "content": "You are a helpful product advisor. You provide gear suggestions for selected travel based on user preferences and relevant products from the catalog."
                           "Be concise, specific, and helpful. Refer to the user by name when appropriate. "
                           "Recommend specific products from the catalog when relevant, providing their name, description, and price if available.",
            },
            {
                "role": "user",
                "content": f"{query}\n{memory_text}\n{product_catalog_text}",
            },
        ]
   
        # Get response from Azure OpenAI
        response = self.azure_client.chat.completions.create(
            model=AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME,
            messages=messages,
            temperature=0.7,
            max_tokens=800,
        )
        
        # Extract response content
        response_content = response.choices[0].message.content
        
        # Store the conversation in memory
        #conversation = [
        #    {"role": "user", "content": query},
        #    {"role": "assistant", "content": response_content}
        #]
        #self.memory.add(conversation, user_id=self.user_id)
        
        return response_content


# Part 4b

Demoing end to end flow with a series of commands

Two agents (1) Travel Assistant and (2) Product Advisor 
AI Search used for Mem0 interaction for both agents seperately
AI Search used for RAG with Product Advisor

In [11]:
import os
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_core.tools import FunctionTool
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat

# --- Environment Variables (Ensure these are set) ---
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME = os.getenv("AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME")

# Create the AzureOpenAIChatCompletionClient with function calling enabled
llm = AzureOpenAIChatCompletionClient(
    azure_deployment= AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME,
    model=AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME,
    api_version= "2025-01-01-preview",
    azure_endpoint= AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
)

# Create instances
travel_assistant = TravelAssistant(name="travel_assistant", user_id="ben_whistler_2025")
product_advisor = ProductAdvisor(user_id="ben_whistler_2025")

# Wrap your methods as FunctionTool
travel_assistant_tool = FunctionTool(travel_assistant.get_response, description="Travel Assistant helping with travel plans.")
product_advisor_tool = FunctionTool(product_advisor.get_product_response, description="Product Advisor helping with product recommendations.")

# --- Agent ---
assistant_agent = AssistantAgent(
     name="Orchestrator",
     description="Orchestrates the travel assistant and product advisor.",
     model_client=llm,
     tools=[product_advisor_tool, travel_assistant_tool],
     system_message="""
     Answer questions asked by the user. 
     Use travel assistant to answer questions regarding locations, places, ski slopes and more. 
     Use product advisor agent to help answer questions regarding gear and clothing. 
     For gear and clothing, only use the product catalog to find relevant products.
     Be concise, specific, to the point and helpful.
     Respond simple, short, clear and to the point.
     When the response is complete and all perspectives are covered, you can respond with TERMINATE.",
    """
)

# --- User Inputs ---
USER_INPUTS = [
     "I am planning our annual winter trip to Whistler.",
     "Can you recommend some good hiking gear for my trip?",
     "which ski slopes are the best for me?",
]

# Define a termination condition 
termination = TextMentionTermination("TERMINATE") 

team = RoundRobinGroupChat(
     participants=[assistant_agent], termination_condition=termination
)

async def run_chat():
    for user_input in USER_INPUTS:
        print(f"User: {user_input}")
        async for response in team.run_stream(task=user_input):
            # Try to extract the message content in a robust way
            content = None
            # If response is an object with .content
            if hasattr(response, "content"):
                content = getattr(response, "content", None)
            # If response is a dict with 'content'
            elif isinstance(response, dict) and "content" in response:
                content = response["content"]
            # If response is an object with .choices (OpenAI style)
            elif hasattr(response, "choices") and response.choices:
                content = response.choices[0].message.content
            # If response is a dict with 'choices'
            elif isinstance(response, dict) and "choices" in response and response["choices"]:
                content = response["choices"][0]["message"]["content"]
            # Fallback: just print the response
            if content:
                print(f"Agent: {content}")
            else:
                print(f"Agent: {response}")

    await llm.close()

# In a Jupyter notebook cell, run:
await run_chat()




Travel Assistant initialized for user ben_whistler_2025
Product Advisor initialized for user ben_whistler_2025 with hybrid search for product catalog.
User: I am planning our annual winter trip to Whistler.
Agent: I am planning our annual winter trip to Whistler.
Agent: That sounds exciting! What specific information do you need for your trip to Whistler? Are you looking for ski slope recommendations, accommodation options, or something else?
Agent: [FunctionCall(id='call_zGsQIwh7IugUZHV9XgYCCNZc', arguments='{"query":"Whistler ski slopes and accommodations"}', name='get_response')]
Including 5 memories in context
Included Memories:
- Planning annual winter ski trip to Whistler
- Likes skiing
- Wants to avoid black and advanced ski slopes due to a past injury
- Enjoys skiing and outdoor activities
- Is an expert skier
Agent: [FunctionExecutionResult(content='Hi! For your annual winter ski trip to Whistler, I recommend focusing on the following slopes and accommodations that suit your p